**ReadMe**

The question 1,2 is answered in part 1

The question 3 is answered in part 2

The question 4 is answered in part 3

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
%cd /content/drive/My\ Drive/Winter\ 2022/CME241

Mounted at /content/drive
/content/drive/My Drive/Winter 2022/CME241


In [ ]:
from rl.markov_process import *
from rl.distribution import *
from rl.markov_decision_process import *
from rl.td import *
from rl.function_approx import *
from typing import Tuple, Sequence, Iterator, List, Dict
from rl.chapter3.simple_inventory_mdp_cap import *
from rl.approximate_dynamic_programming import *
from operator import itemgetter
from rl.policy import *
from rl.monte_carlo import *
import matplotlib.pyplot as plt
import numpy as np
import time
from rl.dynamic_programming import *
from typing import Sequence, Callable, Tuple, Iterator, List
from dataclasses import dataclass

#1. DP & RL for Finite MDP: Simple Inventory Control

In [ ]:
capacity: int = 2
poisson_lambda: float = 1.0
holding_cost: float = 1.0
stockout_cost: float = 10.0
user_gamma: float = 0.9

si_mdp: SimpleInventoryMDPCap = SimpleInventoryMDPCap(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)

##1.1 Dynamic Programming

Policy Iteration

In [ ]:
policy_iteration_result(mdp = si_mdp, gamma = user_gamma)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.894855781630035,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.660960231637507,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.991900091403533,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.660960231637507,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.991900091403533,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991900091403533},
 For State InventoryState(on_hand=0, on_order=0): Do Action 1
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

Value Iteration

In [ ]:
value_iteration_result(mdp = si_mdp, gamma = user_gamma)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.894855194671294,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.66095964467877,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.99189950444479,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.66095964467877,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.99189950444479,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991899504444792},
 For State InventoryState(on_hand=0, on_order=0): Do Action 1
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

## 1.2 Reinforcement Learning

###1.2.1 Monte Carlo

Let's look at the case when we always start at having full inventory on hand.

In [ ]:
si_mdp.non_terminal_states

[NonTerminal(state=InventoryState(on_hand=0, on_order=0)),
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)),
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)),
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)),
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)),
 NonTerminal(state=InventoryState(on_hand=2, on_order=0))]

THus, we will set the initial distirbution to have only the last state

In [ ]:
start_state_distribution = Constant(si_mdp.non_terminal_states[-1])

In [ ]:
qfs: Iterator[QValueFunctionApprox[S, A]] = glie_mc_control(si_mdp,  start_state_distribution, Tabular(), 0.9, lambda n: 1/n)
next(qfs)

def almost_equal_qfs(
    v1: QValueFunctionApprox[S, A],
    v2: QValueFunctionApprox[S, A],
    tolerance: float = 1e-1
) -> bool:
    '''Return whether the two value function tables are within the given
    tolerance of each other.
    '''
    return max(abs(v1.values_map[s] - v2.values_map[s]) for s in v1.values_map) < DEFAULT_TOLERANCE

opt_qf: QValueFunctionApprox[S, A] = converged(qfs, done=almost_equal_qfs)

In [ ]:
opt_qf.values_map

{(NonTerminal(state=InventoryState(on_hand=0, on_order=0)),
  0): -48.10485424176997,
 (NonTerminal(state=InventoryState(on_hand=0, on_order=0)),
  1): -34.905129903961466,
 (NonTerminal(state=InventoryState(on_hand=0, on_order=0)),
  2): -42.40188264324315,
 (NonTerminal(state=InventoryState(on_hand=0, on_order=1)),
  0): -33.91955816407902,
 (NonTerminal(state=InventoryState(on_hand=0, on_order=1)),
  1): -27.676522103892697,
 (NonTerminal(state=InventoryState(on_hand=0, on_order=2)),
  0): -36.19133135362624,
 (NonTerminal(state=InventoryState(on_hand=1, on_order=0)),
  0): -38.05627157227237,
 (NonTerminal(state=InventoryState(on_hand=1, on_order=0)),
  1): -28.678752392142272,
 (NonTerminal(state=InventoryState(on_hand=1, on_order=1)),
  0): -29.00880230308017,
 (NonTerminal(state=InventoryState(on_hand=2, on_order=0)),
  0): -30.006490204283047}

Now that we get optimal q function, we will tyhen try to convert it back into policy and v function

In [ ]:
opt_vf: Mapping[NonTerminal[S], float] = dict()
greedy_policy_dict: Dict[S, A] = dict()
for s in si_mdp.non_terminal_states:
  opt_vf[s] =  max(opt_qf((s, a)) for a in si_mdp.actions(s))
  _, greedy_policy_dict[s.state] = opt_qf.argmax((s, a) for a in si_mdp.actions(s))

In [ ]:
opt_vf, FiniteDeterministicPolicy(greedy_policy_dict)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.905129903961466,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.676522103892697,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -36.19133135362624,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.678752392142272,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.00880230308017,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.006490204283047},
 For State InventoryState(on_hand=0, on_order=0): Do Action 1
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

Check with the value from DP

In [ ]:
value_iteration_result(mdp = si_mdp, gamma = user_gamma)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.894855194671294,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.66095964467877,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.99189950444479,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.66095964467877,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.99189950444479,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991899504444792},
 For State InventoryState(on_hand=0, on_order=0): Do Action 1
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

We will see that the value is different when on_order is 2.

Afterwards, we can create a function.

In [ ]:
Distribution[NonTerminal]

rl.distribution.Distribution[rl.markov_process.NonTerminal]

In [ ]:
def finite_glie_mc_result(mdp: FiniteMarkovDecisionProcess, start_state_distribution: Distribution[NonTerminal], gamma: float, epsilon_as_func_of_episodes: Callable[[int], float] = lambda n: 1/n, tolerance:float = DEFAULT_TOLERANCE, episode_length_tolerance: float=1e-06):
  qfs: Iterator[QValueFunctionApprox[S, A]] = glie_mc_control(mdp,  start_state_distribution, Tabular(), gamma, epsilon_as_func_of_episodes)
  next(qfs)

  def almost_equal_qfs(
      v1: QValueFunctionApprox[S, A],
      v2: QValueFunctionApprox[S, A],
      tolerance: float = tolerance
  ) -> bool:
      '''Return whether the two value function tables are within the given
      tolerance of each other.
      '''
      return max(abs(v1.values_map[s] - v2.values_map[s]) for s in v1.values_map) < tolerance

  opt_qf: QValueFunctionApprox[S, A] = converged(qfs, done=almost_equal_qfs)

  opt_vf: Mapping[NonTerminal[S], float] = dict()
  greedy_policy_dict: Dict[S, A] = dict()
  for s in si_mdp.non_terminal_states:
    opt_vf[s] =  max(opt_qf((s, a)) for a in si_mdp.actions(s))
    _, greedy_policy_dict[s.state] = opt_qf.argmax((s, a) for a in si_mdp.actions(s))

  return opt_vf, FiniteDeterministicPolicy(greedy_policy_dict)

Example with early ending (tolerance is high)

In [ ]:
finite_glie_mc_result(mdp=si_mdp, 
                      start_state_distribution=Choose(si_mdp.non_terminal_states), 
                      gamma=user_gamma,
                      tolerance = 1e-1)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -37.46180438467768,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -29.18641922751893,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -30.562545763100374,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -30.460660034866002,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -30.800628905579266,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -32.47535514053803},
 For State InventoryState(on_hand=0, on_order=0): Do Action 2
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

###1.2.2 SARSA 

Define new glie sarsa function

In [ ]:
si_mdp.actions(NonTerminal(state))

dict_keys([0, 1, 2])

The main difference is taht instead of yielding the function at every update step, we update the decision at the end of the episode.

(Note that the actual updating will still occurring in the online meanner, but we will not print it out.)

In [ ]:
#Note that states in this case mean the start states distribution.
def new_glie_sarsa(
    mdp: MarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    γ: float,
    ϵ_as_func_of_episodes: Callable[[int], float],
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    q: QValueFunctionApprox[S, A] = approx_0
    yield q
    num_episodes: int = 0
    while True:
        num_episodes += 1
        ϵ: float = ϵ_as_func_of_episodes(num_episodes)
        #Start the episode by choosing 1 from the states
        state: NonTerminal[S] = states.sample()
        #Choose 1 action according to the epsilon_greedy_policy (which is indeed also adapted with Q (for the 1-epsilon case))
        #State in the argument is our current state
        #mdp.actions(state) is a dictionary key
        action: A = epsilon_greedy_action(q, state, mdp.actions(state), ϵ=ϵ)
        steps: int = 0

        #Repeat until the state is non-terminal or the episode ends 
        while isinstance(state, NonTerminal) and steps < max_episode_length:
            #Update the state
            #Sample one possible next_state, reward pair associated with the action (which has been decided)
            next_state, reward = mdp.step(state, action).sample()
            if isinstance(next_state, NonTerminal):
                #Guess the action for the next state using the same (current) q.
                next_action: A = epsilon_greedy_action(q, next_state, set(mdp.actions(next_state)), ϵ)
                #After that, we will be able to update the q
                q = q.update([((state, action), reward + γ * q((next_state, next_action)))])
                action = next_action
            else:
                #Because we assume that the last state will have the reward of zero
                q = q.update([((state, action), reward)])
            steps += 1
            state = next_state
        yield q

Add the convergence test into it.

In [ ]:
def finite_glie_sarsa_result(mdp: FiniteMarkovDecisionProcess, start_state_distribution: Distribution[NonTerminal], gamma: float, epsilon_as_func_of_episodes: Callable[[int], float] = lambda n: 1/n, tolerance:float = DEFAULT_TOLERANCE, max_episode_length: float=100):
  
  qfs: Iterator[QValueFunctionApprox[S, A]] = new_glie_sarsa(mdp,
                                                         start_state_distribution,
                                                         Tabular(),
                                                         gamma,
                                                         epsilon_as_func_of_episodes,
                                                         max_episode_length)
  
  next(qfs)

  def almost_equal_qfs(
      v1: QValueFunctionApprox[S, A],
      v2: QValueFunctionApprox[S, A],
      tolerance: float = tolerance
  ) -> bool:
      '''Return whether the two value function tables are within the given
      tolerance of each other.
      '''
      return max(abs(v1.values_map[s] - v2.values_map[s]) for s in v1.values_map) < tolerance
  
  opt_qf: QValueFunctionApprox[S, A] = converged(qfs, done=almost_equal_qfs)

  opt_vf: Mapping[NonTerminal[S], float] = dict()
  greedy_policy_dict: Dict[S, A] = dict()
  for s in si_mdp.non_terminal_states:
    opt_vf[s] =  max(opt_qf((s, a)) for a in si_mdp.actions(s))
    _, greedy_policy_dict[s.state] = opt_qf.argmax((s, a) for a in si_mdp.actions(s))

  return opt_vf, FiniteDeterministicPolicy(greedy_policy_dict)

In [ ]:
finite_glie_sarsa_result(mdp=si_mdp,
                      start_state_distribution=Choose(si_mdp.non_terminal_states),
                      gamma = user_gamma,
                      max_episode_length=1000, tolerance=1e-2)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -27.784916328399795,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -20.836969141233247,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -20.826921289702042,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -21.786121262687033,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -21.973576238178676,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -22.887821070657093},
 For State InventoryState(on_hand=0, on_order=0): Do Action 2
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

###1.2.3 Q-Learning

SARSA is complicated in the sense that we have to adapt the policy at every timestep.

However, for Q-Learning, the algorithm can look more complicated. 

Thus, we will restrict attention on annotating the q-learning function first.

In [ ]:
def q_learning(
    mdp: MarkovDecisionProcess[S, A],
    policy_from_q: PolicyFromQType,
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    γ: float,
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    q: QValueFunctionApprox[S, A] = approx_0
    yield q

    #Different from SARSA, we do not have to adjust epsilon according to the episode. (so we do not have to count the number of epsiode)
    while True:
        #Initialize state according to the start state distribution
        state: NonTerminal[S] = states.sample()
        #The step size will be used for terminating sequence in case the state evolve infinitely
        steps: int = 0
        
        #Notice that we just have to specifiy state
        #The action will not be determined yet
        
        #Note that epsilon is not shown explicitly, but will be used implicitly via policy_from_q (if we choose it to be epsilon-greedy policy)

        #If the state is terminal or we have run sufficiently long step (hard-indicated via max_episode_length)
        while isinstance(state, NonTerminal) and steps < max_episode_length:
            #Find new policy adpated to q
            policy: Policy[S, A] = policy_from_q(q, mdp)
            #Sample action
            action: A = policy.act(state).sample()
            #After the action is realized, we can sample the transition according to the state and the action (and also the mdp)
            next_state, reward = mdp.step(state, action).sample()
            
            #Use greedy return according to q (which is current q)
            #If the next_state is terminal, we set the return after that state to be zero
            next_return: float = max(q((next_state, a)) for a in mdp.actions(next_state)) if isinstance(next_state, NonTerminal) else 0.
            
            #We update the q
            q = q.update([((state, action), reward + γ * next_return)])

            yield q

            steps += 1
            state = next_state

Now, let look into the case when we want the yielding of the q function to be after each episode instead of each step.

We will also use epsilon greedy in each exploration.

In [ ]:
def new_q_learning(
    mdp: MarkovDecisionProcess[S, A],
    policy_from_q: PolicyFromQType,
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    γ: float,
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    q: QValueFunctionApprox[S, A] = approx_0
    yield q

    #Different from SARSA, we do not have to adjust epsilon according to the episode. (so we do not have to count the number of epsiode)
    while True:
        #Initialize state according to the start state distribution
        state: NonTerminal[S] = states.sample()
        #The step size will be used for terminating sequence in case the state evolve infinitely
        steps: int = 0
        
        #Notice that we just have to specifiy state
        #The action will not be determined yet
        
        #Note that epsilon is not shown explicitly, but will be used implicitly via policy_from_q (if we choose it to be epsilon-greedy policy)

        #If the state is terminal or we have run sufficiently long step (hard-indicated via max_episode_length)
        while isinstance(state, NonTerminal) and steps < max_episode_length:
            #Find new policy adpated to q
            policy: Policy[S, A] = policy_from_q(q, mdp)
            #Sample action
            action: A = policy.act(state).sample()
            #After the action is realized, we can sample the transition according to the state and the action (and also the mdp)
            next_state, reward = mdp.step(state, action).sample()
            
            #Use greedy return according to q (which is current q)
            #If the next_state is terminal, we set the return after that state to be zero
            next_return: float = max(q((next_state, a)) for a in mdp.actions(next_state)) if isinstance(next_state, NonTerminal) else 0.
            
            #We update the q
            q = q.update([((state, action), reward + γ * next_return)])

            steps += 1
            state = next_state
        yield q

In [ ]:
def q_learning_epsilon_greedy(
    mdp: MarkovDecisionProcess[S, A],
    epsilon: float,
    start_state_distribution: NTStateDistribution[S],
    initial_func_approx: QValueFunctionApprox[S, A],
    gamma: float,
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:

    def policy_from_q(qf, mdp, epsilon=epsilon):
      return epsilon_greedy_policy(qf, mdp, epsilon)

    return new_q_learning(mdp, policy_from_q, start_state_distribution, initial_func_approx, gamma, max_episode_length)

In [ ]:
hi = q_learning_epsilon_greedy(si_mdp, 1e-1, Choose(si_mdp.non_terminal_states), Tabular(), user_gamma, 200)

In [ ]:
def q_learning_epsilon_greedy_result(mdp: FiniteMarkovDecisionProcess, 
                                     start_state_distribution: Distribution[NonTerminal], 
                                     gamma: float, 
                                     epsilon: float,
                                     tolerance:float = DEFAULT_TOLERANCE,
                                     max_episode_length: float=100):
  
  qfs: Iterator[QValueFunctionApprox[S, A]] = q_learning_epsilon_greedy(mdp,
                                                                        epsilon,
                                                                        start_state_distribution,
                                                                        Tabular(),
                                                                        gamma,
                                                                        max_episode_length)
  
  next(qfs)

  def almost_equal_qfs(
      v1: QValueFunctionApprox[S, A],
      v2: QValueFunctionApprox[S, A],
      tolerance: float = tolerance
  ) -> bool:
      '''Return whether the two value function tables are within the given
      tolerance of each other.
      '''
      return max(abs(v1.values_map[s] - v2.values_map[s]) for s in v1.values_map) < tolerance
  
  opt_qf: QValueFunctionApprox[S, A] = converged(qfs, done=almost_equal_qfs)

  opt_vf: Mapping[NonTerminal[S], float] = dict()
  greedy_policy_dict: Dict[S, A] = dict()
  for s in si_mdp.non_terminal_states:
    opt_vf[s] =  max(opt_qf((s, a)) for a in si_mdp.actions(s))
    _, greedy_policy_dict[s.state] = opt_qf.argmax((s, a) for a in si_mdp.actions(s))

  return opt_vf, FiniteDeterministicPolicy(greedy_policy_dict)

In [ ]:
q_learning_epsilon_greedy_result(si_mdp, Choose(si_mdp.non_terminal_states), user_gamma, 0.1, 1e-2, 200)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -23.708881697939493,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -16.477160943574702,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -16.64774178411582,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -17.4760098810234,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -17.713356855566282,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -18.656796562971696},
 For State InventoryState(on_hand=0, on_order=0): Do Action 2
 For State InventoryState(on_hand=0, on_order=1): Do Action 1
 For State InventoryState(on_hand=0, on_order=2): Do Action 0
 For State InventoryState(on_hand=1, on_order=0): Do Action 1
 For State InventoryState(on_hand=1, on_order=1): Do Action 0
 For State InventoryState(on_hand=2, on_order=0): Do Action 0)

#2. ADP & RL for Non-Finite MDP: Discrete Asset Allocation

In [ ]:
@dataclass(frozen=True)
class AssetAllocDiscrete:
    risky_return_distributions: Sequence[Distribution[float]]
    riskless_returns: Sequence[float]
    utility_func: Callable[[float], float]
    risky_alloc_choices: Sequence[float]
    feature_functions: Sequence[Callable[[Tuple[float, float]], float]]
    dnn_spec: DNNSpec
    initial_wealth_distribution: Distribution[float]

    def time_steps(self) -> int:
        return len(self.risky_return_distributions)

    def uniform_actions(self) -> Choose[float]:
        return Choose(self.risky_alloc_choices)

    def get_mdp(self, t: int) -> MarkovDecisionProcess[float, float]:
        """
        State is Wealth W_t,
        Action is investment in risky asset = x_t
        """

        distr: Distribution[float] = self.risky_return_distributions[t]
        rate: float = self.riskless_returns[t]
        alloc_choices: Sequence[float] = self.risky_alloc_choices
        steps: int = self.time_steps()
        utility_f: Callable[[float], float] = self.utility_func

        class AssetAllocMDP(MarkovDecisionProcess[float, float]):
            #wealth is state
            #alloc is action
            #both are stored as float

            #step will indicate how current state and action affect the next distribution of (state and reward) in tuple form
            def step(self, wealth: NonTerminal[float], alloc: float) -> SampledDistribution[Tuple[State[float], float]]:
                #state reward sampler
                def sr_sampler_func(wealth=wealth, alloc=alloc) -> Tuple[State[float], float]:
                    next_wealth: float = alloc * (1 + distr.sample()) + (wealth.state - alloc) * (1 + rate)
                    #if it is going to be terminated, give a reward.
                    #otherwsie, do nothing
                    reward: float = utility_f(next_wealth) if t == steps - 1 else 0.
                    next_state: State[float] = Terminal(next_wealth) if t == steps - 1 else NonTerminal(next_wealth)
                    return (next_state, reward)
                return SampledDistribution(sampler=sr_sampler_func, expectation_samples=100)

            #Possible actions
            def actions(self, wealth: NonTerminal[float]) -> Sequence[float]:
                return alloc_choices

        return AssetAllocMDP()

    def get_qvf_func_approx(self) -> DNNApprox[Tuple[NonTerminal[float], float]]:
        adam_gradient: AdamGradient = AdamGradient(learning_rate=0.1, decay1=0.9, decay2=0.999)
        #List of function fetaures: Nonterminal State x Action ---> features
        ffs: List[Callable[[Tuple[NonTerminal[float], float]], float]] = []
        for f in self.feature_functions:
            def this_f(pair: Tuple[NonTerminal[float], float], f=f) -> float:
                return f((pair[0].state, pair[1]))
            ffs.append(this_f)

        return DNNApprox.create(feature_functions=ffs, dnn_spec=self.dnn_spec, adam_gradient=adam_gradient)

    #Get states distribution at time t not codnitioning on doing anything
    def get_states_distribution(self, t: int) -> SampledDistribution[NonTerminal[float]]:
        
        #Action distribution
        actions_distr: Choose[float] = self.uniform_actions()

        def states_sampler_func() -> NonTerminal[float]:
            wealth: float = self.initial_wealth_distribution.sample()
            for i in range(t):
                distr: Distribution[float] = self.risky_return_distributions[i]
                rate: float = self.riskless_returns[i]
                #sample the ation
                alloc: float = actions_distr.sample()
                #sample the wealth
                wealth = alloc * (1 + distr.sample()) + (wealth - alloc) * (1 + rate)
            return NonTerminal(wealth)

        return SampledDistribution(states_sampler_func)


    def backward_induction_qvf(self) -> Iterator[QValueFunctionApprox[float, float]]:

        #Initilaize function approximtaion
        init_fa: DNNApprox[Tuple[NonTerminal[float], float]] = self.get_qvf_func_approx()

        mdp_f0_mu_triples: Sequence[Tuple[MarkovDecisionProcess[float, float], DNNApprox[Tuple[NonTerminal[float], float]], SampledDistribution[NonTerminal[float]]]] = [(self.get_mdp(i), init_fa, self.get_states_distribution(i)) for i in range(self.time_steps())]

        num_state_samples: int = 300
        error_tolerance: float = 1e-6

        return back_opt_qvf(mdp_f0_mu_triples=mdp_f0_mu_triples, γ=1.0, num_state_samples=num_state_samples, error_tolerance=error_tolerance)

    def get_vf_func_approx(self, ff: Sequence[Callable[[NonTerminal[float]], float]]) -> DNNApprox[NonTerminal[float]]:

        adam_gradient: AdamGradient = AdamGradient(learning_rate=0.1, decay1=0.9,decay2=0.999)
        return DNNApprox.create(feature_functions=ff, dnn_spec=self.dnn_spec, adam_gradient=adam_gradient)

    def backward_induction_vf_and_pi(self, ff: Sequence[Callable[[NonTerminal[float]], float]]) -> Iterator[Tuple[ValueFunctionApprox[float], DeterministicPolicy[float, float]]]:

        init_fa: DNNApprox[NonTerminal[float]] = self.get_vf_func_approx(ff)

        mdp_f0_mu_triples: Sequence[Tuple[MarkovDecisionProcess[float, float], DNNApprox[NonTerminal[float]], SampledDistribution[NonTerminal[float]]]] = [(self.get_mdp(i), init_fa, self.get_states_distribution(i)) for i in range(self.time_steps())]

        num_state_samples: int = 100
        error_tolerance: float = 1e-3

        return back_opt_vf_and_policy(mdp_f0_mu_triples=mdp_f0_mu_triples, γ=1.0, num_state_samples=num_state_samples, error_tolerance=error_tolerance)

In [ ]:
from pprint import pprint

steps: int = 2
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_stdev: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: - np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(2 / 3 * base_alloc, 4 / 3 * base_alloc,11)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = [lambda _: 1., lambda w_x: w_x[0], lambda w_x: w_x[1], lambda w_x: w_x[1] * w_x[1]]
dnn: DNNSpec = DNNSpec(neurons=[], bias=False, hidden_activation=lambda x: x, hidden_activation_deriv=lambda y: np.ones_like(y), output_activation=lambda x: - np.sign(a) * np.exp(-x),output_activation_deriv=lambda y: -y)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_stdev)

aad: AssetAllocDiscrete = AssetAllocDiscrete(
        risky_return_distributions=risky_ret,
        riskless_returns=riskless_ret,
        utility_func=utility_function,
        risky_alloc_choices=alloc_choices,
        feature_functions=feature_funcs,
        dnn_spec=dnn,
        initial_wealth_distribution=init_wealth_distr
    )

In [ ]:
    it_qvf: Iterator[QValueFunctionApprox[float, float]] = \
        aad.backward_induction_qvf()

    print("Backward Induction on Q-Value Function")
    print("--------------------------------------")
    print()
    for t, q in enumerate(it_qvf):
        print(f"Time {t:d}")
        print()
        opt_alloc: float = max(
            ((q((NonTerminal(init_wealth), ac)), ac) for ac in alloc_choices),
            key=itemgetter(0)
        )[1]
        val: float = max(q((NonTerminal(init_wealth), ac))
                         for ac in alloc_choices)
        print(f"Opt Risky Allocation = {opt_alloc:.3f}, Opt Val = {val:.3f}")
        print("Optimal Weights below:")
        for wts in q.weights:
            pprint(wts.weights)
        print()

    print("Analytical Solution")
    print("-------------------")
    print()

    for t in range(steps):
        print(f"Time {t:d}")
        print()
        left: int = steps - t
        growth: float = (1 + r) ** (left - 1)
        alloc: float = base_alloc / growth
        vval: float = - np.exp(- excess * excess * left / (2 * var)
                               - a * growth * (1 + r) * init_wealth) / a
        bias_wt: float = excess * excess * (left - 1) / (2 * var) + \
            np.log(np.abs(a))
        w_t_wt: float = a * growth * (1 + r)
        x_t_wt: float = a * excess * growth
        x_t2_wt: float = - var * (a * growth) ** 2 / 2

        print(f"Opt Risky Allocation = {alloc:.3f}, Opt Val = {vval:.3f}")
        print(f"Bias Weight = {bias_wt:.3f}")
        print(f"W_t Weight = {w_t_wt:.3f}")
        print(f"x_t Weight = {x_t_wt:.3f}")
        print(f"x_t^2 Weight = {x_t2_wt:.3f}")
        print()

Backward Induction on Q-Value Function
--------------------------------------

Time 0

Opt Risky Allocation = 1.400, Opt Val = -0.291
Optimal Weights below:
array([[ 0.04992729,  1.14322909,  0.06191486, -0.02253809]])

Time 1

Opt Risky Allocation = 1.400, Opt Val = -0.328
Optimal Weights below:
array([[-0.0056287 ,  1.06953962,  0.07161126, -0.02476941]])

Analytical Solution
-------------------

Time 0

Opt Risky Allocation = 1.402, Opt Val = -0.291
Bias Weight = 0.045
W_t Weight = 1.145
x_t Weight = 0.064
x_t^2 Weight = -0.023

Time 1

Opt Risky Allocation = 1.500, Opt Val = -0.328
Bias Weight = 0.000
W_t Weight = 1.070
x_t Weight = 0.060
x_t^2 Weight = -0.020



In [ ]:
q.weights[0].weights

array([[-0.0056287 ,  1.06953962,  0.07161126, -0.02476941]])

##2.2 Reinforcement Learning

In the previous section, we used backward induction approximate dynamic programming in order to solve the problem. However, in many cases, we do not know the actual dynamics of the system.

In other words, we may not knwo the change in distribution of the outcome when we take some action instead of another. Unluckily, sometimes, the only way to be able to do so is to try the action.

However, there is usually still some reqularity. In other words, we do not have to learn all possible occurence (We do not have to learn all state action pair to get the full picture of the Q function.)

What we do is to learn from enough state action pair and hope that we can interpolate what we have learned into other cases.

For this problem specfically, let us first create an MDP for this problem. (Note that we cannot do the backward induction, because we cannot suddenly start at the last timestep without passing through all the previous time steps)

In [ ]:
#First, we create a class for the state
# Note that the current state will contain wealth and time
#Let say that wealth is stored directly as float and time is stored directly as int for simplicity

@dataclass(frozen=True)
class WealthTimeState:
    wealth: float
    time: int

#Next, we create the action state. In this case, action will also be float, but we will save it in a class for systematicity

@dataclass(frozen=True)
class AllocationState:
    allocation: float

Next, we will create the MDP

In [ ]:
@dataclass(frozen=True)
class DiscreteAssetAllocationMDP(MarkovDecisionProcess[WealthTimeState, AllocationState]):
    risky_return_distributions: Sequence[Distribution[float]]
    riskless_returns: Sequence[float]
    utility_func: Callable[[float], float]
    risky_alloc_choices: Sequence[float]
    feature_functions: Sequence[Callable[[Tuple[float, float]], float]]
    dnn_spec: DNNSpec
    initial_wealth_distribution: Distribution[float]

    def step(self, non_terminal_state: NonTerminal[WealthTimeState], allocation: AllocationState) -> SampledDistribution[Tuple[State[WealthTimeState], float]]:

        def sample_next_state_reward(non_terminal_state=non_terminal_state,allocation=allocation) -> Tuple[State[WealthTimeState], float]:
            t = non_terminal_state.state.time
            wealth = non_terminal_state.state.wealth
            next_wealth: float = allocation.allocation * (1 + self.risky_return_distributions[t].sample()) + (wealth - allocation.allocation) * (1 + self.riskless_returns[t])
            reward: float = self.utility_func(next_wealth) if t == steps - 1 else 0.
            next_state: State[float] = Terminal(WealthTimeState(wealth=next_wealth, time=t+1)) if t == steps - 1 else NonTerminal(WealthTimeState(wealth=next_wealth, time=t+1))
            return (next_state, reward)
        return SampledDistribution(sampler=sample_next_state_reward, expectation_samples=100)

    #Possible actions
    def actions(self, non_terminal_state: NonTerminal[WealthTimeState]) -> Sequence[float]:
      return [AllocationState(allocation=allocation) for allocation in alloc_choices]

In [ ]:
steps: int = 2
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_stdev: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: - np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(2 / 3 * base_alloc, 4 / 3 * base_alloc,11)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = [lambda _: 1., lambda w_x: w_x[0], lambda w_x: w_x[1], lambda w_x: w_x[1] * w_x[1]]
dnn: DNNSpec = DNNSpec(neurons=[], bias=False, hidden_activation=lambda x: x, hidden_activation_deriv=lambda y: np.ones_like(y), output_activation=lambda x: - np.sign(a) * np.exp(-x),output_activation_deriv=lambda y: -y)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_stdev)

Hi = DiscreteAssetAllocationMDP(risky_ret, riskless_ret, utility_function, alloc_choices, feature_funcs, dnn, init_wealth_distr)

In [ ]:
#The start state can be seen as when the wealth is the initial wealth and time = 0
def get_start_states_distribution(init_wealth_distr) -> SampledDistribution[NonTerminal[WealthTimeState]]:
  def states_sampler_func() -> NonTerminal[float]:
    wealth: float = init_wealth_distr.sample()
    return NonTerminal(WealthTimeState(wealth=wealth, time=1))
  return SampledDistribution(states_sampler_func)


In [ ]:
def get_qvf_func_approx(feature_funcs, dnn) ->  DNNApprox[Tuple[NonTerminal[WealthTimeState], AllocationState]]:

        adam_gradient: AdamGradient = AdamGradient(learning_rate=0.1, decay1=0.9, decay2=0.999)
        ffs: List[Callable[[Tuple[NonTerminal[WealthTimeState], AllocationState]], float]] = []
        for f in feature_funcs:
            def this_f(pair: Tuple[NonTerminal[WealthTimeState], AllocationState], f=f) -> float:
                return f((pair[0].state.wealth, pair[1].allocation))
            ffs.append(this_f)

        return DNNApprox.create(feature_functions=ffs,dnn_spec=dnn,adam_gradient=adam_gradient)


In [ ]:
Hey = q_learning_epsilon_greedy(mdp=Hi, epsilon=0.1, start_state_distribution= get_start_states_distribution(init_wealth_distr),
    initial_func_approx = get_qvf_func_approx(feature_funcs, dnn),
    gamma = 1,
    max_episode_length = 10)

In [ ]:
next(Hey)

DNNApprox(feature_functions=[<function get_qvf_func_approx.<locals>.this_f at 0x7f963b127320>, <function get_qvf_func_approx.<locals>.this_f at 0x7f963b1278c0>, <function get_qvf_func_approx.<locals>.this_f at 0x7f963b1275f0>, <function get_qvf_func_approx.<locals>.this_f at 0x7f963b1277a0>], dnn_spec=DNNSpec(neurons=[], bias=False, hidden_activation=<function <lambda> at 0x7f9646bd28c0>, hidden_activation_deriv=<function <lambda> at 0x7f9646bd2680>, output_activation=<function <lambda> at 0x7f9646bd2ef0>, output_activation_deriv=<function <lambda> at 0x7f963b127050>), regularization_coeff=0.0, weights=[Weights(adam_gradient=AdamGradient(learning_rate=0.1, decay1=0.9, decay2=0.999), time=4, weights=array([[ 0.62199951, -0.50970893,  0.73662396,  0.26585708]]), adam_cache1=array([[-0.00448753, -0.00381631, -0.00897506, -0.01795012]]), adam_cache2=array([[3.08323590e-05, 2.99194135e-05, 1.23329436e-04, 4.93317743e-04]]))])

#3. RL for Non-Finite MDP: Banker Problem

Since the action of borrowing and investing is to be done in the end of the day, we let the state be the observation at the end of the day.

1. Payback: We borrow $y$ yesterday and have to pay it back $(1+R)y$
2. Deposit: Let assume that the incoming deposit is a random variable $D$
3. Withdrawal: Let assume that the incoming withdrawal is a random varaible $W$
4. Penalty: If the start cash is less than $C$, we have to pay a penalty of $ \cot\left({\frac{\pi c}{2C}}\right)$.
5. Spot Prize Growth: from 1 to $S$
6. Number of Investment: $v$
7. Borrow: To get more cash at the end of the day
8. Postponed Withdraw: $\delta$
9. Total balance: $b$


Assume that we start the day with cash = $c$, debt = $y$, and postponed withdrawal = $\delta$

Before the start of the business day, the amount of money we have will be (after the penalty has been paid)
\begin{align*}
  c \ge 0
\end{align*}
Since $c$ can be adjusted by borrowing, this constraint can be staisfied with sufficient amount of borrowing.

After the morning, the amount of cash we have will be
\begin{align*}
  c + D
\end{align*}
Note that this value will be non-negative

After the afternoon, the amount of cash we have will be
\begin{align*}
  \max(c + D - W - \delta, 0)
\end{align*}
and we postpone the withdrawal of $\max(W + \delta - D - c +p(c), 0)$ to tomorrow.


At the evening, the amount of cash we have will be
\begin{align*}
  \max(c + D - W - \delta, 0)
\end{align*}
and the total balance becomes $b+D-W$.

Thus, the total wealth will be cash $ = v S + \max(c - p(c) + D - W - \delta, 0) - y(1-R)$.

The decision we have to make is to select $v$ and $y$ such that the next day welath become $c = \text{cash} - v + b$.

## 2.1 State

We consider the state to consist of the following quantity.
1. balance: $b \ge 0$ 
2. postpone: $\delta \ge 0$
3. total wealth: $x$ can be negative
4. time: $t \ge 0$

The transition rule for time is $t' = t+1$ independent from every action or variable.

$b$ will determine $W$ since $W$ cannot exceed $b$. Let $D$ be independent. 

Let $S$ be independent.

Thus, given that the previous day we have $S = (b, \delta, x, t)$.

We selected action $A = (v, y)$ where $v$ is investment, and $y$ is amount borrowed.

1. $c = x-v+y$ (At that time)
2. $c = c - p(c)$ (At that time)
3. $D$ is drawn from $f_D$ (Before the morning)
4. $W$ is drawn from $f_W(b)$ (We draw it before updating $b$ since we assume that people will not depsoit and withdraw in the same day). (Before the morning)
5. $c = c + D$ (After the morning)
6. $b = b + D$ (After the morning)
7. $W = W + \delta$ (Before the afternoon)
8. $\delta = \max(W-c, 0)$ (After the afternoon)
9. $c = \max(c-W, 0)$ (After the afternoon)
10. $b = b - W$ (After the afternoon)
11. $S$ is drawn from $f_S$ (After the afternoon)
12. $x = vS + c - (1+R)y$
13. $t = t+1$

To sum up,
1. $c = x-v+y$ 
2. $c = c - p(c) \ge 0$
3. $D, W, S$ are drawn from $f_D, f_W(b), f_S$, respectively.
4. $b' = b + D - W$
5. $c, \delta' = \max(c+D-W-\delta, 0), \max(W+\delta-c-D, 0)$
6. $x' = vS + c - (1+R)y$
7. $t' = t+1$

The state

\begin{align*}
  \mathcal{S} &= \left\{ (b, \delta, x, t) \in \mathbf{R}_{+} \times \mathbf{R}_{+} \times \mathbf{R} \times \mathbf{N}_0 \vert t 
  \le T\right
  \}\\
  \mathcal{N} &= \left\{ (b, \delta, x, t) \in \mathbf{R}_{+} \times \mathbf{R}_{+} \times \mathbf{R} \times \mathbf{N}_0 \vert t 
  \le T-1\right
  \}\\
  \mathcal{T} &= \mathbf{R}_{+} \times \mathbf{R}_{+} \times \mathbf{R} \times \{T\}
\end{align*}

The action (which is dependent on the state)
\begin{align*}
  \mathcal{A}(x) &= \left\{ (v, y) \in \mathbf{R}_{+} \times \mathbf{R}_{+} \vert x-v+y \ge C \text{ or } x-v+y \ge \cot\left({\frac{\pi x-v+y}{2C}}\right)\right \}
\end{align*}

The reward will be $0$ at every transotion, except for at the transition from time $T-1$ to $T$, where the reward will be the final $x$. This problem can then be solved by backward induction with ADP.

In [111]:
@dataclass(frozen=True)
class State:
  b: float
  delta: float
  x: float
  t: int

@dataclass(frozen=True)
class Action:
  v: float
  y: float